# <h1 align="center"> DOPP Ex3
<h5 align="center"> Group 08: Simhandl, Wanecek and Beck

In [ ]:
import os
path= ''
currentdirectory = os.path.abspath(path)
os.chdir(currentdirectory)

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

# Kinematics-motion-dataset

source:
https://www.kaggle.com/yasserh/kinematics-motion-data

In [ ]:
# import raw data
df=pd.read_csv('datasets\kinematics-motion-data.csv', sep=',', encoding='latin-1')
df.info()
df.head()

## Explore kinematics-motion-dataset

In [ ]:
# Date, Time and Username are no universal features to predict the activites Running and Walking so we can drop them first
df = df.drop(columns=['username','date','time'],axis=1)

In [ ]:
df.shape
df.isna().sum()

In [ ]:
# get descriptive statistics
df.describe()

In [ ]:
# explor dataset using pandas profiling
from pandas_profiling import ProfileReport
prof = ProfileReport(df)#, minimal = True)
prof.to_file(output_file='explore_kinematics-motion-dataset.html')

# Communities and Crime dataset

source:
https://archive.ics.uci.edu/ml/datasets/Communities+and+Crime

In [ ]:
# import raw data
df=pd.read_csv('datasets\communities_crime.csv', sep=',', encoding='latin-1')

df.info()
df.head()

## Explore communities and crime dataset

In [ ]:
df.shape
df.isna().sum()

In [ ]:
# get descriptive statistics
df.describe()

In [ ]:
# explor dataset using pandas profiling
from pandas_profiling import ProfileReport
prof = ProfileReport(df, minimal = True)
prof.to_file(output_file='explore_communities-crime-dataset.html')